### -----------------------------------------------Function-----------------------------------------------

In [5]:
import pandas as pd

def process_data(df_ref_path, unique_rows_path):
    # Lire le fichier en utilisant un point-virgule comme séparateur
    df = pd.read_csv(df_ref_path, encoding='ISO-8859-1', delimiter=';') 

    # Splitter la première colonne en deux colonnes en utilisant l+a première virgule comme séparateur
    df[['Country', 'Company_Suffixes']] = df.iloc[:,0].str.split(',', n=1, expand=True) 

    # Enlever les guillemets doubles dans la colonne 'Company_Suffixes' 
    df['Company_Suffixes'] = df['Company_Suffixes'].str.replace('"', '') 
    df = df.iloc[:, -2:] 
    df_suffix = df 

    df_suffix.to_csv('country_suffix.csv', index=False) 

    unique_rows = pd.read_csv(unique_rows_path) 
    df_suffix = pd.read_csv('country_suffix.csv') 

    # Convertir toutes les valeurs de 'Name_company' en chaînes de caractères
    unique_rows['Name_company'] = unique_rows['Name_company'].astype(str) 

    df_suffix['Country'] = df_suffix['Country'].str.lower()  
    unique_rows['Country'] = unique_rows['Country'].str.lower() 

    unique_rows['Name_company'] = unique_rows['Name_company'].str.lower() 

    # Fusionner les deux dataframes sur la colonne 'Country'
    merged_df = pd.merge(unique_rows, df_suffix, on='Country', how='inner')

    # Ensuite, vous pouvez définir une fonction qui vérifie si un suffixe existe dans le nom de l'entreprise.
    import re

    def check_suffix(row):
        company_name = row['Name_company']
        suffixes = row['Company_Suffixes']

        # Vérifier si l'un des mots dans le nom de l'entreprise est dans la liste des suffixes
        # Diviser sur les espaces et les points
        company_words = re.split('[ .]', company_name)
        for word in company_words:
            if word in suffixes:
                return True
        return False


    # Ensuite, appliquez cette fonction à chaque ligne de merged_df pour créer la nouvelle colonne.
    merged_df['suffix_in_name'] = merged_df.apply(check_suffix, axis=1)

    new_df = merged_df[['Name_company','Country', 'suffix_in_name']]

    return new_df

In [6]:
new_df = process_data('C:/Users/haykel.bargougui/STAGE/TrueFalse/df_ref.csv', 'C:/Users/haykel.bargougui/STAGE/TrueFalse/unique_rows.csv')
new_df.to_csv('True_False_Suffix.csv', index=False)

In [7]:
new_df

,Name_company,Country,suffix_in_name
0,10x genomics inc. class a common stock,united states,True
1,1life healthcare inc,united states,True
2,1st source corporation common stock,united states,True
3,"22nd century group, inc.",united states,True
4,2seventy bio inc,united states,True
...,...,...,...
15865,messer tehnogas ad,serbia,True
15866,pomoravlje ad nis,serbia,True
15867,putevi ad uzice,serbia,True
15868,societe libanaise des ciments blancs sal,lebanon,False


# ***************************************************************************

In [14]:
import pandas as pd
import re

def process_data(df_ref_path, unique_rows_path):
    # Lire le fichier en utilisant un point-virgule comme séparateur
    df = pd.read_csv(df_ref_path, encoding='ISO-8859-1', delimiter=';') 

    # Splitter la première colonne en deux colonnes en utilisant l+a première virgule comme séparateur
    df[['Country', 'Company_Suffixes']] = df.iloc[:,0].str.split(',', n=1, expand=True) 

    # Enlever les guillemets doubles dans la colonne 'Company_Suffixes' 
    df['Company_Suffixes'] = df['Company_Suffixes'].str.replace('"', '').str.lower()
    df = df.iloc[:, -2:] 
    df_suffix = df 

    df_suffix.to_csv('country_suffix.csv', index=False) 

    unique_rows = pd.read_csv(unique_rows_path) 
    df_suffix = pd.read_csv('country_suffix.csv') 

    # Convertir toutes les valeurs de 'Name_company' en chaînes de caractères
    unique_rows['Name_company'] = unique_rows['Name_company'].astype(str) 

    df_suffix['Country'] = df_suffix['Country'].str.lower()  
    unique_rows['Country'] = unique_rows['Country'].str.lower() 

    unique_rows['Name_company'] = unique_rows['Name_company'].str.lower() 

    # Fusionner les deux dataframes sur la colonne 'Country'
    merged_df = pd.merge(unique_rows, df_suffix, on='Country', how='inner')

    # Ensuite, vous pouvez définir une fonction qui vérifie si un suffixe existe dans le nom de l'entreprise.
    def check_suffix(row):
        company_name = row['Name_company']
        suffixes = row['Company_Suffixes']

        # Initialiser une liste pour stocker les suffixes trouvés
        suffixes_found = []

        # Diviser sur les espaces et sur les points qui sont suivis d'un espace
        # Cela permet de conserver les points qui sont directement liés à un mot
        company_words = re.split(' |\.\s', company_name)
        for word in company_words:
            if word and word in suffixes:  # Ignorer les chaînes vides
                suffixes_found.append(word)
        return suffixes_found
    

    # Appliquer la fonction check_suffix pour créer une nouvelle colonne 'suffix_concerned'.
    merged_df['suffix_concerned'] = merged_df.apply(check_suffix, axis=1)

    # Créer une colonne 'suffix_in_name' pour vérifier si 'suffix_concerned' est vide ou non
    merged_df['suffix_in_name'] = merged_df['suffix_concerned'].apply(lambda x: bool(x))

    new_df = merged_df[['Name_company', 'Country', 'suffix_in_name', 'suffix_concerned']]

    return new_df


In [15]:
new_df = process_data('C:/Users/haykel.bargougui/STAGE/TrueFalse/df_ref.csv', 'C:/Users/haykel.bargougui/STAGE/TrueFalse/unique_rows.csv')
def remove_single_letter_words(word_list):
    return [word for word in word_list if len(word) > 1]

new_df['suffix_concerned'] = new_df['suffix_concerned'].apply(remove_single_letter_words)

new_df.to_csv('True_False_Suffix.csv', index=False)

In [16]:
new_df

,Name_company,Country,suffix_in_name,suffix_concerned
0,10x genomics inc. class a common stock,united states,True,[inc]
1,1life healthcare inc,united states,True,[inc]
2,1st source corporation common stock,united states,True,[corporation]
3,"22nd century group, inc.",united states,True,[inc.]
4,2seventy bio inc,united states,True,[inc]
...,...,...,...,...
15865,messer tehnogas ad,serbia,True,[ad]
15866,pomoravlje ad nis,serbia,True,[ad]
15867,putevi ad uzice,serbia,True,[ad]
15868,societe libanaise des ciments blancs sal,lebanon,False,[]


In [17]:
filtered_df = new_df[new_df['suffix_in_name'] == True]
filtered_df

,Name_company,Country,suffix_in_name,suffix_concerned
0,10x genomics inc. class a common stock,united states,True,[inc]
1,1life healthcare inc,united states,True,[inc]
2,1st source corporation common stock,united states,True,[corporation]
3,"22nd century group, inc.",united states,True,[inc.]
4,2seventy bio inc,united states,True,[inc]
...,...,...,...,...
15862,industria sulfurica s.a.,uruguay,True,[s.a.]
15863,grupo financiero improsa s.a.,costa rica,True,[s.a.]
15865,messer tehnogas ad,serbia,True,[ad]
15866,pomoravlje ad nis,serbia,True,[ad]


In [19]:
grouped_df = new_df.explode('suffix_concerned').groupby('suffix_concerned')['Country'].unique().reset_index()
grouped_df['Country'] = grouped_df['Country'].apply(lambda x: ', '.join(x))
grouped_df.rename(columns={'Country': 'Countries_with_same_suffix'}, inplace=True)

print(grouped_df.head())


  suffix_concerned       Countries_with_same_suffix
0              a.s                           turkey
1               ac                    united states
2       acceptance                    united states
3               ad  united states, bulgaria, serbia
4               ag                      switzerland


In [20]:
# Filtrer les lignes avec 'suffix_in_name' == True
filtered_df = new_df[new_df['suffix_in_name'] == True]

print(filtered_df.head())


                             Name_company        Country  suffix_in_name  \
0  10x genomics inc. class a common stock  united states            True   
1                    1life healthcare inc  united states            True   
2     1st source corporation common stock  united states            True   
3                22nd century group, inc.  united states            True   
4                        2seventy bio inc  united states            True   

  suffix_concerned  
0            [inc]  
1            [inc]  
2    [corporation]  
3           [inc.]  
4            [inc]  


In [25]:
final_df = filtered_df.copy()
final_df['suffix_concerned_str'] = final_df['suffix_concerned'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')

final_df = pd.merge(final_df, grouped_df, left_on='suffix_concerned_str', right_on='suffix_concerned', how='left')
final_df.to_csv('country_with_same_suffix.csv', index=False)
final_df

,Name_company,Country,suffix_in_name,suffix_concerned_x,suffix_concerned_str,suffix_concerned_y,Countries_with_same_suffix
0,10x genomics inc. class a common stock,united states,True,[inc],inc,inc,"united states, australia, philippines"
1,1life healthcare inc,united states,True,[inc],inc,inc,"united states, australia, philippines"
2,1st source corporation common stock,united states,True,[corporation],corporation,corporation,"united states, canada, south africa, namibia"
3,"22nd century group, inc.",united states,True,[inc.],inc.,inc.,"united states, puerto rico"
4,2seventy bio inc,united states,True,[inc],inc,inc,"united states, australia, philippines"
...,...,...,...,...,...,...,...
9217,industria sulfurica s.a.,uruguay,True,[s.a.],s.a.,s.a.,"chile, peru, morocco, ecuador, uruguay, costa ..."
9218,grupo financiero improsa s.a.,costa rica,True,[s.a.],s.a.,s.a.,"chile, peru, morocco, ecuador, uruguay, costa ..."
9219,messer tehnogas ad,serbia,True,[ad],ad,ad,"united states, bulgaria, serbia"
9220,pomoravlje ad nis,serbia,True,[ad],ad,ad,"united states, bulgaria, serbia"


# Quelques staistics sur le resultat :

In [14]:
import pandas as pd

# Lire les fichiers csv
df1 = pd.read_csv('unique_rows.csv')
df2 = pd.read_csv('country_suffix.csv')

In [15]:
# Obtenir les countries uniques dans chaque dataframe
unique_countries_df1 = df1['Country'].unique()
unique_countries_df2 = df2['Country'].unique()

# Trouver les countries qui sont dans df2 mais pas dans df1
not_in_df1 = set(unique_countries_df2) - set(unique_countries_df1)

# Convertir l'ensemble en une liste
countries_not_in_df1 = list(not_in_df1)

# Afficher les résultats
print(countries_not_in_df1)

['malta ', 'Haiti', 'Ethiopia', 'Virginia', 'netherlands ', 'Maryland', 'Idaho', 'Alaska', 'Kentucky', 'greece ', 'North Macedonia', 'Hawaii', 'Colorado', 'Monaco', 'Oregon', 'Montana', 'Rhode Island', 'California', 'iceland ', 'ireland ', 'Ukraine', 'Vermont', 'estonia ', 'portugal ', 'Dominican Republic', 'finland ', 'Iran', 'Georgia', 'Mississippi', 'Texas', 'italy ', 'North Dakota', 'Guatemala', 'norway ', 'Pennsylvania', 'france ', 'lithuania ', 'liechtenstein ', 'Nicaragua', 'denmark ', 'romania', 'austria ', 'Nevada', 'Arizona', 'germany  ', 'South Dakota', 'Oklahoma', 'New Mexico', 'New Hampshire', 'hungary ', 'Indiana', 'Delaware', 'spain ', 'belgium ', 'West Virginia', 'Wisconsin', 'New York', 'Washington', 'Albania', 'South Carolina', 'District of Columbia', 'Iowa', 'Alabama', 'croatia ', 'Utah', 'poland ', 'Kosovo', 'Korea South', 'Missouri', 'Connecticut', 'Louisiana', 'bulgaria', 'Illinois', 'latvia ', 'Michigan', 'Bosnia and Herzegovina', 'European Union and European Eco

In [16]:
import pandas as pd

# Lire le fichier csv
df = pd.read_csv('True_False_Suffix.csv')

# Grouper par 'Country' et 'suffix_in_name', et compter les occurrences
counts = df.groupby(['Country', 'suffix_in_name']).size()

# Créer un DataFrame à partir de la série 'counts'
counts_df = counts.reset_index(name='counts')

# Pivoter le DataFrame pour avoir 'Country' comme index et 'suffix_in_name' comme colonnes
pivot_df = counts_df.pivot(index='Country', columns='suffix_in_name', values='counts')

# Remplacer les valeurs NaN par 0 (si nécessaire)
pivot_df = pivot_df.fillna(0)

# Filtrer les pays où le compte de False est plus grand que le compte de True

In [17]:
filtered_df = pivot_df[pivot_df[False] > pivot_df[True]]

# Afficher les résultats
print(filtered_df)

suffix_in_name         False   True
Country                            
argentina               32.0    1.0
australia              286.0   46.0
cambodia                 5.0    2.0
canada                1177.0    1.0
chile                  198.0    0.0
colombia                37.0    0.0
costa rica               1.0    0.0
ecuador                 20.0    0.0
egypt                   73.0    6.0
ghana                   11.0    2.0
hong kong              146.0   75.0
indonesia               34.0    2.0
japan                 2056.0  142.0
jordan                  55.0    2.0
kazakhstan               1.0    0.0
lebanon                  2.0    0.0
malaysia               398.0    1.0
morocco                 32.0    0.0
namibia                 17.0    0.0
nepal                    3.0    0.0
new zealand             25.0    1.0
nigeria                 58.0    0.0
oman                     4.0    0.0
pakistan               192.0    9.0
peru                   114.0    0.0
philippines             21.0

In [18]:
filtered_df = pivot_df[pivot_df[True] > pivot_df[False]]

# Afficher les résultats
print(filtered_df)

suffix_in_name  False    True
Country                      
brazil           13.0   111.0
bulgaria         12.0    19.0
china           235.0   801.0
guam              0.0     1.0
israel           13.0    76.0
mexico            3.0    19.0
romania           1.0     3.0
serbia            0.0     3.0
switzerland      12.0   102.0
turkey           63.0   227.0
united states   646.0  4665.0
